# Ingesta de Datos Precios al Consumidor

In [56]:
import requests
import glob
import os

import unidecode
import pandas as pd

## Loading datos

In [24]:
ids = {
    
    2008:"ce3ee3f9-64d9-47cb-8410-3a5efc5c556d",
    2009:"c3368d09-548c-4237-aeae-fabe44fb4fab",
    2010:"13d4f76c-89ee-4b28-9a44-f36d4d12490a",
    2011:"81aec091-3864-42b3-8fc7-40af01596c46",
    2012:"f21bfea7-7c7d-4719-a30b-83ab93c5d778",
    2013:"8ffb24c7-f96e-4313-9a1f-de7ccc80f5ab",
    2014:"e6c1ec11-75f8-48c6-9436-404667d1574a",
    2015:"10c5d3a5-2156-4b25-8204-29de777beb0b",
    2016:"424470c6-5357-4d2b-9755-8da62864286b",
    2017:"0d1ed43e-a3d0-4cfc-8c17-f004f33bbfbb",
    2018:"6e2fdb16-6095-434f-9118-c81bdf8ea0b6",
    2019:"0ca89699-9a0a-40d3-867e-85ee2b2a1d4f",
    2020:"d2a6ca13-09b8-42e2-a5fb-22345dc79d00",
    2021:"9078be17-bb39-414d-bc56-0f8c0e994ce9",
    2022:"46580cb7-605f-44be-9402-40ba5891c878",
    2023:"2c238cbd-2508-46f6-9a52-6aad18f45a9a",
    2024:"557c130a-7775-46d0-b245-6a3481d9208e"
}

In [27]:
dataframes = []
for year,id in ids.items():
    url = f"https://datos.gob.cl/api/action/datastore_search?&resource_id={id}&limit=999999999"
    # Realizamos GET
    response = requests.get(url)

    # Si es exitoso concatenar, en caso contrario print
    if response.status_code == 200:
        # Parseando el JSON
        json_data = response.json()
        # Extrayendo datos de interés
        records = json_data["result"]["records"]
        df = pd.DataFrame(records)
        dataframes.append(df)
    else:
        print(f"Fallo al intentar descargar datos de {year}. Status code: {response.status_code}")

Fallo al intentar descargar datos de 2020. Status code: 404
Fallo al intentar descargar datos de 2022. Status code: 404
Fallo al intentar descargar datos de 2023. Status code: 404
Fallo al intentar descargar datos de 2024. Status code: 404


In [ ]:
# Agregando faltantes
csv_files = glob.glob(os.path.join("data", "*.csv"))

In [40]:
# Loop over each CSV file path
for file in csv_files:
    # Read each CSV file into a DataFrame
    df = pd.read_csv(file)
    
    # Append the DataFrame to the list
    dataframes.append(df)

In [94]:
final_df = pd.concat(dataframes, ignore_index=True)

## Procesado

### Merge columnas año

In [95]:
# Vemos un cambio en el orden de las columnas
print(final_df.columns.tolist())

['_id', '\ufeffAño', 'Mes', 'Semana', 'Fecha inicio', 'Fecha término', 'ID región', 'Región', 'Sector', 'Tipo de punto monitoreo', 'Grupo', 'Producto', 'Variedad', 'Calidad', 'Unidad', 'Precio mínimo', 'Precio máximo', 'Precio promedio', 'Año']


In [96]:
# Pequeña revisión para cerciorarnos de que las columnas año se complementan (una es nula y la otra no)
mask = (final_df['Año'].isnull() & final_df['\ufeffAño'].notnull()) | (final_df['\ufeffAño'].isnull() & final_df['Año'].notnull())
complementary_check = mask.all()

# Output result
print(f"Are the columns complementary? {complementary_check}")

Are the columns complementary? True


In [97]:
# Mergeando año
final_df['Año'] = final_df['Año'].fillna(final_df['\ufeffAño'])

# Dropeando la extraña
final_df.drop(columns=['\ufeffAño'], inplace=True)

### Estandarización nombre columnas

In [98]:
def standardize_column_names(df):
    """
    Función para estandarizar nombre de columnas de la df.
    """
    # Convertir a lowercase
    df.columns = df.columns.str.lower()
    
    # Reemplazar guión bajo
    df.columns = df.columns.str.replace(' ', '_')
    
    # Quitar carácteres especiales
    df.columns = df.columns.map(lambda x: unidecode.unidecode(x))
    
    # Reemplazamos ano por anyo
    df.columns = df.columns.str.replace('ano', 'anyo', regex=False)
    
    return df

In [116]:
processed_df = standardize_column_names(final_df)

In [117]:
processed_df.sample()

,_id,mes,semana,fecha_inicio,fecha_termino,id_region,region,sector,tipo_de_punto_monitoreo,grupo,producto,variedad,calidad,unidad,precio_minimo,precio_maximo,precio_promedio,anyo
3429304,NaN,3,9,2023-02-27,2023-03-03,13,Región Metropolitana de Santiago,Centro,Supermercado,Lácteos - Huevos - Margarinas,Leche en Polvo Entera,NaN,NaN,$/bolsa 800 grs,6850,6850,"6850,00",2023.0


### Nuevo id

In [118]:
# Primero ordenamos por anyo, mes y semana
processed_df.sort_values(["anyo","mes","semana"], inplace=True)

In [119]:
# Al ver valores nulos, se dropea y se resetea el index para que funcione como id
processed_df = processed_df.drop(columns=['_id'])
processed_df = processed_df.reset_index(drop=True)

### Data type

In [120]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6656149 entries, 0 to 6656148
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   mes                      int64  
 1   semana                   int64  
 2   fecha_inicio             object 
 3   fecha_termino            object 
 4   id_region                int64  
 5   region                   object 
 6   sector                   object 
 7   tipo_de_punto_monitoreo  object 
 8   grupo                    object 
 9   producto                 object 
 10  variedad                 object 
 11  calidad                  object 
 12  unidad                   object 
 13  precio_minimo            object 
 14  precio_maximo            object 
 15  precio_promedio          object 
 16  anyo                     float64
dtypes: float64(1), int64(3), object(13)
memory usage: 863.3+ MB


In [121]:
# Fechas
processed_df['fecha_inicio'] = pd.to_datetime(processed_df['fecha_inicio'], format='%Y-%m-%d', errors='coerce')
processed_df['fecha_termino'] = pd.to_datetime(processed_df['fecha_termino'], format='%Y-%m-%d', errors='coerce')

In [122]:
processed_df.nunique()

mes                           12
semana                        53
fecha_inicio                 192
fecha_termino                192
id_region                      9
region                         9
sector                        47
tipo_de_punto_monitoreo        7
grupo                          7
producto                     158
variedad                     117
calidad                       14
unidad                        21
precio_minimo              10218
precio_maximo              10308
precio_promedio            95227
anyo                          17
dtype: int64

In [123]:
# Categorías
columns_to_convert = ['region', 'tipo_de_punto_monitoreo', 'grupo', 'calidad']

for column in columns_to_convert:
    processed_df[column] = processed_df[column].astype('category')

In [124]:
# Numéricas
processed_df.filter(like="precio").sample(10)

,precio_minimo,precio_maximo,precio_promedio
119568,699,799,"749,00"
5122835,1390,1390,"1390,00"
5037364,6490,6490,"6490,00"
5184242,1790,1790,"1790,00"
1194335,2398,2398,"2398,00"
5569749,5990,5990,"5990,00"
1613018,7980,7980,"7980,00"
894527,6090,6090,"6090,00"
4508721,1450,1450,"1450,00"
3275705,1500,1500,"1500,00"


In [125]:
processed_df["precio_diff"] = processed_df["precio_minimo"] != processed_df["precio_maximo"]

In [126]:
processed_df["precio_diff"].value_counts(dropna=False)

precio_diff
False    5981139
True      675010
Name: count, dtype: int64

In [127]:
processed_df.drop(columns=["precio_diff"],inplace=True)

In [128]:
# Reemplazamos las comas de columnas precio y convertimos a numérico
columns_to_fix = ['precio_minimo', 'precio_maximo', 'precio_promedio']

for column in columns_to_fix:
    processed_df[column] = processed_df[column].str.replace(',', '.', regex=False) 
    processed_df[column] = pd.to_numeric(processed_df[column], errors='coerce')

In [129]:
# Cambiamos año a int
processed_df["anyo"] = processed_df["anyo"].astype("int16")

In [130]:
processed_df.sample()

,mes,semana,fecha_inicio,fecha_termino,id_region,region,sector,tipo_de_punto_monitoreo,grupo,producto,variedad,calidad,unidad,precio_minimo,precio_maximo,precio_promedio,anyo
3493598,2,7,2023-02-13,2023-02-17,5,Región de Valparaíso,Viña del Mar,Feria libre,Frutas,Durazno,Elegant Lady,Primera,$/kilo,NaN,NaN,1200.0,2023


## Guardado del nuevo df

In [134]:
processed_df.to_parquet("processed/processed_precios.parquet")